PIPELINE FOR INTEGRATION OF ORDERS AND DETAILS

In [1]:
from order_details import load_order_details
from order import load_order
from exporting import export, export_data

import os
import pathlib
import os.path as Path
import pandas as pd
import warnings
from datetime import date, datetime

warnings.filterwarnings('ignore')

In [2]:
#INPUTS
INPUT_DATE = None
# INPUT_DATE = '2021-01-01'

if not INPUT_DATE is None:
    INPUT_DATE =  datetime.strptime(INPUT_DATE, '%Y-%m-%d').date()

ORDER_DETAILS_FILE = "order_details.csv"

if INPUT_DATE is None:
    date_base = date.today()
else:
    date_base = INPUT_DATE

print(f"Pipeline is running on {date_base}")

Pipeline is running on 2022-07-26


In [3]:
#BASE PATHS
PATH_BASE       = pathlib.Path(f"{os.getcwd()}").parent
PATH_DATA       = f"{PATH_BASE}/data"
PATH_DETAILS    = f"{PATH_DATA}/{ORDER_DETAILS_FILE}"
PATH_POSTGRES   = f"{PATH_DATA}/postgres"
PATH_CSV        = f"{PATH_DATA}/csv"
PATH_RESULT     = f"{PATH_DATA}/result"

EXTRACTING DATA

In [4]:
df_tables = load_order()

Extracting data from the PostgreSQL database...
Connecting to the PostgreSQL database...
PostgreSQL: selecting all tables ...
PostgreSQL: extracting data from tables...
 1/13 us_states
 2/13 customers
 3/13 orders
 4/13 employees
 5/13 shippers
 6/13 categories
 7/13 products
 8/13 suppliers
 9/13 region
 10/13 territories
 11/13 employee_territories
 12/13 customer_demographics
 13/13 customer_customer_demo
Database connection closed.


In [6]:
df_details = load_order_details(PATH_DETAILS)

Exctracting data from .csv file.
 ORDER_DETAILS: finished process


TRANSFORM DATA

In [7]:
def exists_dir(path, create_dir=True):
    if not Path.isdir(path):
        if create_dir: 
            os.mkdir(path)
            # print(f"created {path}...")
        return False
    else: 
        return True

def exists_file(file_path):
    return Path.exists(file_path)


In [8]:
#variable for exporting process
export_process = True

In [9]:
date_today = str(date_base)
file_datename = date_base.strftime('%s')

#POSTGRESS
exists_dir(PATH_POSTGRES)

for table_name, df_table in df_tables:
    PATH_TABLE = f"{PATH_POSTGRES}/{table_name}"
    exists_dir(PATH_TABLE)
    
    PATH_TABLE_DATE = f"{PATH_TABLE}/{date_today}"
    exists_dir(PATH_TABLE_DATE)

    #name the exporting file
    POSTGRES_FILE_NAME = f"{PATH_TABLE_DATE}/{file_datename}.parquet"
    
    #save on path 
    df_table.to_parquet(POSTGRES_FILE_NAME)
    
    #valid process for POSTGRES
    created_postgres_file = exists_file(POSTGRES_FILE_NAME)
    
    if not created_postgres_file:
        export_process = False
        print(f"{table_name}: Could not export file to {POSTGRES_FILE_NAME} path.")

#CSV
exists_dir(PATH_CSV)

for table_name, df_detail in df_details:
    PATH_CSV_DATE = f"{PATH_CSV}/{date_today}"
    exists_dir(PATH_CSV_DATE)

    CSV_FILE_NAME = f"{PATH_CSV_DATE}/{file_datename}.parquet"
            
    #save on path 
    df_detail.to_parquet(CSV_FILE_NAME)

    #valid process for CSV
    created_csv_file =  exists_file(CSV_FILE_NAME)

    if not created_csv_file:
        export_process = False
        print(f"details: Could not export file to {CSV_FILE_NAME} path.")

EXPORT

In [10]:
if len(df_tables) > 0 and len(df_details) > 0:
    #exists CSV path and FILE of the date
    if not exists_dir(PATH_CSV_DATE, False):
        continue_process = False
        print(f"Path of {PATH_CSV_DATE} have not been created, it means that process of exctracting have not been completed successfully.")
        
    if not exists_file(CSV_FILE_NAME):
        continue_process = False
        print(f"File on path {CSV_FILE_NAME} does not exists, it means that process of exctracting have not been completed successfully.")

    if export_process:
        export(date_base)

else:
    print("Exporting process of orders and orders details have not been completed successfully")

Loading data...
 Importing rows 0 to 51 for table us_states...
  Data imported successfull for table us_states.
 Importing rows 51 to 100 for table employee_territories...
  Data imported successfull for table employee_territories.
 No rows to be imported for table customer_customer_demo...
  Table imported successfull on customer_customer_demo.
 Importing rows 100 to 106 for table shippers...
  Data imported successfull for table shippers.
 Importing rows 106 to 183 for table products...
  Data imported successfull for table products.
 No rows to be imported for table customer_demographics...
  Table imported successfull on customer_demographics.
 Importing rows 183 to 274 for table customers...
  Data imported successfull for table customers.
 Importing rows 274 to 283 for table employees...
  Data imported successfull for table employees.
 Importing rows 283 to 336 for table territories...
  Data imported successfull for table territories.
 Importing rows 336 to 340 for table region

In [11]:
df = export_data()

FILE_RESULT = f"{PATH_RESULT}-{date_today}.csv"
df.to_csv(FILE_RESULT, index=False)

if exists_file(FILE_RESULT):
    print(f"Result exported on {FILE_RESULT}")
else:
    print(f"It was not possible to generate the file on {FILE_RESULT}")


Exporting local data to the PostgreSQL database...
 Connecting to the PostgreSQL database...
 2155 results of the query...
 Database connection closed.
Result exported on /home/leonardo/Documents/GitHub/Data_Engineering-Techindicium/data/result-2022-07-26.csv
